In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import time
import urllib3
import json

In [2]:
import os

url = 'https://www.bcn.cl/leychile/navegar/imprimir?idNorma=242302&idVersion=2022-04-29'

In [3]:
## Leyendo archivos

In [4]:
path = 'Textos/'
filelist = os.listdir(path)

In [5]:
filelist.sort()

In [6]:
filelist

['.ipynb_checkpoints',
 'Capitulo 01.txt',
 'Capitulo 02.txt',
 'Capitulo 03.txt',
 'Capitulo 04.txt',
 'Capitulo 05.txt',
 'Capitulo 06.txt',
 'Capitulo 07.txt',
 'Capitulo 08.txt',
 'Capitulo 09.txt',
 'Capitulo 10.txt',
 'Capitulo 11.txt',
 'Capitulo 12.txt',
 'Capitulo 13.txt',
 'Capitulo 14.txt',
 'Capitulo 15.txt',
 'Disposiciones transitorias.txt',
 'Promulgacion.txt']

Eliminando el capitulo de promulgación

In [7]:
filelist.remove('Promulgacion.txt')

In [8]:
filelist

['.ipynb_checkpoints',
 'Capitulo 01.txt',
 'Capitulo 02.txt',
 'Capitulo 03.txt',
 'Capitulo 04.txt',
 'Capitulo 05.txt',
 'Capitulo 06.txt',
 'Capitulo 07.txt',
 'Capitulo 08.txt',
 'Capitulo 09.txt',
 'Capitulo 10.txt',
 'Capitulo 11.txt',
 'Capitulo 12.txt',
 'Capitulo 13.txt',
 'Capitulo 14.txt',
 'Capitulo 15.txt',
 'Disposiciones transitorias.txt']

In [9]:
texto_total = ''

for file in filelist[1:]:
    with open(path+file, 'r', encoding = 'utf-8') as f:
        text = f.read()
        if text[-1:] != '\n':
            text = text+'\n'
        else:
            text = text
    
    texto_total = texto_total + text

serie = texto_total.split('\n')

In [10]:
df = pd.DataFrame({'texto':serie})
df['texto'] = df['texto'].str.strip()

In [11]:
df['capitulo'] = np.where(df['texto'].str.contains('Preámbulo'),df['texto'],np.nan)
df['capitulo'] = np.where(df['texto'].str.contains('DISPOSICIONES TRANSITORIAS'),df['texto'],df['capitulo'])
df['capitulo'] = np.where(df['texto'].str.contains('Capítulo'),df['texto'],df['capitulo'])
df['capitulo'] = np.where(df['texto'].str.contains('DECIMOSEXTA'),np.nan,df['capitulo'])
df['capitulo'] = np.where(df['texto'].str.contains('Los procesos iniciados, de oficio o a petición de parte, o que se iniciaren en la Corte Suprema'),np.nan,df['capitulo'])
df['capitulo'] = np.where(df['texto'].str.contains('Las contiendas de competencia actualmente trabadas ante la Corte Suprema'),np.nan,df['capitulo'])
df['capitulo'] = np.where(df['texto'].str.contains('En lo no previsto en este Capítulo, serán aplicables'),np.nan,df['capitulo'])


#Para asegurar que aparezca el capitulo IV
df['articulo'] = np.where(df['texto'].str.contains('Artículo'),df['texto'],np.nan)
df['texto'] = np.where(df['texto']=='', np.nan, df['texto'])
df['texto'] = np.where(df['texto']==df['articulo'], np.nan, df['texto'])
df['texto'] = np.where(df['texto']==df['capitulo'], np.nan, df['texto'])
# Agregar numeral y ordinal (??)
df['nro_articulo'] = df['articulo'].str.split('.- ', expand = True)[0]
df['txt_articulo'] = df['articulo'].str.split('.- ', expand = True)[1]

df['capitulo'].fillna(method='pad', inplace=True)
df['nro_articulo'].fillna(method='pad', inplace=True)



In [12]:
df['texto'] = np.where(df['texto'].isna(), df['txt_articulo'], df['texto'])

In [13]:
df.dropna(subset = 'texto', inplace = True)

In [14]:
df['nro_capitulo'] = df['capitulo'].str.split(':', expand = True)[0]
df['nro_capitulo'] = df['nro_capitulo'].str.replace('# ','')
df['titulo_capitulo'] = df['capitulo'].str.split(':', expand = True)[1]
df['titulo_capitulo'] = df['titulo_capitulo'].str.strip()

df['titulo_capitulo'] = np.where(df['titulo_capitulo'].isna(), df['nro_capitulo'], df['titulo_capitulo'])

In [15]:
df = df[['nro_capitulo','titulo_capitulo','nro_articulo','texto']]
df.rename(columns = {'nro_capitulo':'capitulo'}, inplace = True)

In [16]:
df.groupby(['capitulo','titulo_capitulo']).count()

,,nro_articulo,texto
capitulo,titulo_capitulo,,
Capítulo I,BASES DE LA INSTITUCIONALIDAD,25,25
Capítulo II,NACIONALIDAD Y CIUDADANIA,33,33
Capítulo III,DE LOS DERECHOS Y DEBERES CONSTITUCIONALES,125,125
Capítulo IV,GOBIERNO,96,96
Capítulo IX,SERVICIO ELECTORAL Y JUSTICIA ELECTORAL,18,18
Capítulo V,CONGRESO NACIONAL,179,179
Capítulo VI,PODER JUDICIAL,30,30
Capítulo VII,MINISTERIO PÚBLICO,19,19
Capítulo VIII,TRIBUNAL CONSTITUCIONAL,50,50


In [17]:
df

,capitulo,titulo_capitulo,nro_articulo,texto
4,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°,Las personas nacen libres e iguales en dignida...
5,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°,La familia es el núcleo fundamental de la soci...
6,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°,El Estado reconoce y ampara a los grupos inter...
7,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°,El Estado está al servicio de la persona human...
8,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°,Es deber del Estado resguardar la seguridad na...
...,...,...,...,...
1705,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,VIGESIMA,"El procedimiento de solicitud, la exención de ..."
1706,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,VIGESIMA,Estarán impedidos de solicitar el retiro a que...
1707,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,VIGESIMA,Quienes hubieren hecho ejercicio del derecho e...
1708,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,VIGESIMA,"Sin perjuicio de lo anterior, quienes hubieren..."


In [18]:
list(df['nro_articulo'].value_counts().index)

['VIGESIMA',
 'Artículo 19',
 'Artículo 93',
 'Artículo 140',
 'Artículo 143',
 'Artículo 63',
 'Artículo 32',
 'Artículo 52',
 'Artículo 53',
 'Artículo 54',
 'Artículo 142',
 'Artículo 57',
 'Artículo 65',
 'Artículo 130',
 'Artículo 113',
 'Artículo 51',
 'Artículo 38 bis',
 'Artículo 78',
 'Artículo 92',
 'Artículo 136',
 'Artículo 118',
 'Artículo 60',
 'Artículo 131',
 'La Corte Suprema nominará, en conformidad a la letra c) del Artículo 92, los abogados indicados en la medida que se vayan generando las vacantes correspondientes. No obstante, el primero de ellos será nombrado por tres años, el segundo por seis años y el tercero por nueve años. El que haya sido nombrado por tres años podrá ser reelegido.',
 'Artículo 64',
 'Artículo 95',
 'Artículo 124',
 'Artículo 111',
 'Artículo 133',
 'Artículo 77',
 'Artículo 134',
 'Artículo 115',
 'Artículo 10',
 'Artículo 11',
 'Artículo 67',
 'Artículo 109',
 'Artículo 135',
 'Artículo 128',
 'Artículo 129',
 'Artículo 132',
 'Artículo 96

## Notas:
* Corregir artículos